In [69]:
import os
import psycopg2
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from dotenv import load_dotenv
# Load the environment variables
load_dotenv()

True

In [70]:
payload = '''SELECT f.open, f.high, f.low, f.close, f.hist_close, f.tick_vol, t.*
				FROM PUBLIC.financial_data AS f
				JOIN PUBLIC.technical_data AS t
				ON f.date_time=t.date_time
                ORDER BY f.date_time ASC;'''

In [71]:
database='fx_tradingDB'
user=os.getenv('POSTGRE_USERNAME')
password=os.getenv('POSTGRE_PASSWORD')
port=os.getenv('POSTGRE_PORT')

conn = psycopg2.connect(database=database,
                        user=user,
                        password=password,
                        port=port)

cur = conn.cursor()
cur.execute(payload)
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
cur.close()
conn.close()

In [72]:
usd_df = pd.DataFrame(rows, columns=columns)
if 'tech_key' in usd_df.columns: # (To run multiple times without error)
    usd_df.drop(columns='tech_key', inplace=True)

usd_df

,open,high,low,close,hist_close,tick_vol,momentum_kama,others_cr,trend_ema_fast,trend_ema_slow,...,volatility_dch,volatility_dcl,volatility_dcm,volatility_kcc,volatility_kch,volatility_kcl,volume_obv,volume_vpt,volume_vwap,date_time
0,1.0237,1.0239,1.0213,1.0217,0.0,403,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008-06-11 08:00:00
1,1.0219,1.0227,1.0214,1.022,1.0217,391,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0249,1.0197,403.0,0.0,0.0,2008-06-11 09:00:00
2,1.0221,1.0222,1.0196,1.0196,1.022,399,0.0,0.029362826661438746,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0241166666666666,1.0202166666666668,794.0,0.1148086522462255,0.0,2008-06-11 10:00:00
3,1.0197,1.0205,1.0173,1.0192,1.0196,480,0.0,-0.20553978663012673,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0237666666666667,1.0194333333333334,395.0,-0.822177649123602,0.0,2008-06-11 11:00:00
4,1.0193,1.0195,1.0184,1.0189,1.0192,239,0.0,-0.24469022217871172,0.0,0.0,...,0.0,0.0,0.0,0.0,1.023375,1.0185250000000001,-85.0,-1.0104867899631484,0.0,2008-06-11 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.38337,1.38366,1.38214,1.3822,1.38337,1842,1.3820589433254304,35.39884506215132,1.3821317876205197,1.3818474110242265,...,1.38487,1.3807,1.3827850000000002,1.3819106666666667,1.3831186666666668,1.3807026666666666,3137599.0,1751.4771914233474,1.382098723524962,2024-07-26 19:00:00
99996,1.38219,1.38365,1.38196,1.38332,1.3822,2192,1.3820607165423122,35.28433003817169,1.3821422818327476,1.3818735287261357,...,1.38487,1.3807,1.3827850000000002,1.3820286666666668,1.3833456666666666,1.3807116666666666,3135757.0,1749.9193001867297,1.3821653919222028,2024-07-26 20:00:00
99997,1.38332,1.38395,1.38292,1.38376,1.38332,1914,1.3821256484040352,35.39395125770775,1.3823234692430941,1.381980674746422,...,1.38487,1.3807,1.3827850000000002,1.3821843333333335,1.3836013333333332,1.3807673333333335,3137949.0,1751.6954830835612,1.3822584391906283,2024-07-26 21:00:00
99998,1.38376,1.38398,1.38306,1.38308,1.38376,1436,1.3822544686762286,35.43701673681121,1.382544473974926,1.3821124766170576,...,1.38487,1.3807,1.3827850000000002,1.3824279999999998,1.383866,1.38099,3139863.0,1752.3042793129225,1.3823760486183698,2024-07-26 22:00:00


In [73]:
usd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   open                     100000 non-null  object        
 1   high                     100000 non-null  object        
 2   low                      100000 non-null  object        
 3   close                    100000 non-null  object        
 4   hist_close               100000 non-null  object        
 5   tick_vol                 100000 non-null  object        
 6   momentum_kama            100000 non-null  object        
 7   others_cr                100000 non-null  object        
 8   trend_ema_fast           100000 non-null  object        
 9   trend_ema_slow           100000 non-null  object        
 10  trend_ichimoku_a         100000 non-null  object        
 11  trend_ichimoku_b         100000 non-null  object        
 12  trend_ichimoku_ba

In [74]:
usd_df = usd_df.apply(pd.to_numeric)
usd_df['date_time'] = pd.to_datetime(usd_df['date_time'])
usd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   open                     100000 non-null  float64       
 1   high                     100000 non-null  float64       
 2   low                      100000 non-null  float64       
 3   close                    100000 non-null  float64       
 4   hist_close               100000 non-null  float64       
 5   tick_vol                 100000 non-null  float64       
 6   momentum_kama            100000 non-null  float64       
 7   others_cr                100000 non-null  float64       
 8   trend_ema_fast           100000 non-null  float64       
 9   trend_ema_slow           100000 non-null  float64       
 10  trend_ichimoku_a         100000 non-null  float64       
 11  trend_ichimoku_b         100000 non-null  float64       
 12  trend_ichimoku_ba

In [83]:
usd_df.shape

(100000, 31)

In [82]:
# the outliers
usd_df[(np.abs(stats.zscore(usd_df.drop(columns='date_time'))) >= 2).any(axis=1)]['date_time'].to_list()

[Timestamp('2008-06-11 08:00:00'),
 Timestamp('2008-06-11 09:00:00'),
 Timestamp('2008-06-11 10:00:00'),
 Timestamp('2008-06-11 11:00:00'),
 Timestamp('2008-06-11 12:00:00'),
 Timestamp('2008-06-11 13:00:00'),
 Timestamp('2008-06-11 14:00:00'),
 Timestamp('2008-06-11 15:00:00'),
 Timestamp('2008-06-11 16:00:00'),
 Timestamp('2008-06-11 17:00:00'),
 Timestamp('2008-06-11 18:00:00'),
 Timestamp('2008-06-11 19:00:00'),
 Timestamp('2008-06-11 20:00:00'),
 Timestamp('2008-06-11 21:00:00'),
 Timestamp('2008-06-11 22:00:00'),
 Timestamp('2008-06-11 23:00:00'),
 Timestamp('2008-06-12 00:00:00'),
 Timestamp('2008-06-12 01:00:00'),
 Timestamp('2008-06-12 02:00:00'),
 Timestamp('2008-06-12 03:00:00'),
 Timestamp('2008-06-12 04:00:00'),
 Timestamp('2008-06-12 05:00:00'),
 Timestamp('2008-06-12 06:00:00'),
 Timestamp('2008-06-12 07:00:00'),
 Timestamp('2008-06-12 08:00:00'),
 Timestamp('2008-06-12 09:00:00'),
 Timestamp('2008-06-12 11:00:00'),
 Timestamp('2008-06-12 12:00:00'),
 Timestamp('2008-06-

In [53]:
if 'high' in usd_df.columns:
    usd_df.drop(columns = ['high', 'low'], inplace=True)

In [54]:
usd_df['year'] = usd_df['date_time'].dt.year
usd_df['month'] = usd_df['date_time'].dt.month
usd_df['day'] = usd_df['date_time'].dt.day
usd_df['hour'] = usd_df['date_time'].dt.hour
usd_df['day_of_week'] = usd_df['date_time'].dt.day_of_week


usd_df

,open,close,hist_close,tick_vol,momentum_kama,others_cr,trend_ema_fast,trend_ema_slow,trend_ichimoku_a,trend_ichimoku_b,...,volatility_kcl,volume_obv,volume_vpt,volume_vwap,date_time,year,month,day,hour,day_of_week
0,1.02370,1.02170,0.00000,403.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,2008-06-11 08:00:00,2008,6,11,8,2
1,1.02190,1.02200,1.02170,391.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.022600,...,1.019700,403.0,0.000000,0.000000,2008-06-11 09:00:00,2008,6,11,9,2
2,1.02210,1.01960,1.02200,399.0,0.000000,0.029363,0.000000,0.000000,0.000000,1.022600,...,1.020217,794.0,0.114809,0.000000,2008-06-11 10:00:00,2008,6,11,10,2
3,1.01970,1.01920,1.01960,480.0,0.000000,-0.205540,0.000000,0.000000,0.000000,1.021750,...,1.019433,395.0,-0.822178,0.000000,2008-06-11 11:00:00,2008,6,11,11,2
4,1.01930,1.01890,1.01920,239.0,0.000000,-0.244690,0.000000,0.000000,0.000000,1.020600,...,1.018525,-85.0,-1.010487,0.000000,2008-06-11 12:00:00,2008,6,11,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.38337,1.38220,1.38337,1842.0,1.382059,35.398845,1.382132,1.381847,1.382520,1.381295,...,1.380703,3137599.0,1751.477191,1.382099,2024-07-26 19:00:00,2024,7,26,19,4
99996,1.38219,1.38332,1.38220,2192.0,1.382061,35.284330,1.382142,1.381874,1.382520,1.381295,...,1.380712,3135757.0,1749.919300,1.382165,2024-07-26 20:00:00,2024,7,26,20,4
99997,1.38332,1.38376,1.38332,1914.0,1.382126,35.393951,1.382323,1.381981,1.382520,1.381540,...,1.380767,3137949.0,1751.695483,1.382258,2024-07-26 21:00:00,2024,7,26,21,4
99998,1.38376,1.38308,1.38376,1436.0,1.382254,35.437017,1.382544,1.382112,1.382520,1.381540,...,1.380990,3139863.0,1752.304279,1.382376,2024-07-26 22:00:00,2024,7,26,22,4


### Cyclical Transformations
Features like month and hour are cyclical by their nature. It is important that their cyclical nature be captured in the dataset. For example, the model will interprete 23:00 as far from 00:00 which is not the reality.

In [55]:
usd_df['month_sin'] = np.sin(2 * np.pi * usd_df['month'] / 12)
usd_df['month_cos'] = np.sin(2 * np.pi * usd_df['month'] / 12)

usd_df['hour_sin'] = np.sin(2 * np.pi * usd_df['hour'] / 24)
usd_df['hour_cos'] = np.sin(2 * np.pi * usd_df['hour'] / 24)

usd_df['day_sin'] = np.sin(2 * np.pi * usd_df['day'] / 31)
usd_df['day_cos'] = np.sin(2 * np.pi * usd_df['day'] / 31)

usd_df['day_of_week_sin'] = np.sin(2 * np.pi * usd_df['day_of_week'] / 7)
usd_df['day_of_week_cos'] = np.sin(2 * np.pi * usd_df['day_of_week'] / 7)
usd_df.drop(columns=['year', 'month', 'day', 'hour',
                     'day_of_week'], inplace=True)

usd_df.head()

,open,close,hist_close,tick_vol,momentum_kama,others_cr,trend_ema_fast,trend_ema_slow,trend_ichimoku_a,trend_ichimoku_b,...,volume_vwap,date_time,month_sin,month_cos,hour_sin,hour_cos,day_sin,day_cos,day_of_week_sin,day_of_week_cos
0,1.0237,1.0217,0.0000,403.0,0.0,0.000000,0.0,0.0,0.0,0.00000,...,0.0,2008-06-11 08:00:00,1.224647e-16,1.224647e-16,8.660254e-01,8.660254e-01,0.790776,0.790776,0.974928,0.974928
1,1.0219,1.0220,1.0217,391.0,0.0,0.000000,0.0,0.0,0.0,1.02260,...,0.0,2008-06-11 09:00:00,1.224647e-16,1.224647e-16,7.071068e-01,7.071068e-01,0.790776,0.790776,0.974928,0.974928
2,1.0221,1.0196,1.0220,399.0,0.0,0.029363,0.0,0.0,0.0,1.02260,...,0.0,2008-06-11 10:00:00,1.224647e-16,1.224647e-16,5.000000e-01,5.000000e-01,0.790776,0.790776,0.974928,0.974928
3,1.0197,1.0192,1.0196,480.0,0.0,-0.205540,0.0,0.0,0.0,1.02175,...,0.0,2008-06-11 11:00:00,1.224647e-16,1.224647e-16,2.588190e-01,2.588190e-01,0.790776,0.790776,0.974928,0.974928
4,1.0193,1.0189,1.0192,239.0,0.0,-0.244690,0.0,0.0,0.0,1.02060,...,0.0,2008-06-11 12:00:00,1.224647e-16,1.224647e-16,1.224647e-16,1.224647e-16,0.790776,0.790776,0.974928,0.974928


In [56]:
corr_mat = usd_df.corr()
corr_mat


,open,close,hist_close,tick_vol,momentum_kama,others_cr,trend_ema_fast,trend_ema_slow,trend_ichimoku_a,trend_ichimoku_b,...,volume_vwap,date_time,month_sin,month_cos,hour_sin,hour_cos,day_sin,day_cos,day_of_week_sin,day_of_week_cos
open,1.000000,0.999949,0.999726,0.319086,0.997164,0.999999,0.996654,0.992777,0.992858,0.998974,...,0.996045,0.785418,0.066335,0.066335,-0.001145,-0.001145,-0.005813,-0.005813,-0.001819,-0.001819
close,0.999949,1.000000,0.999676,0.319069,0.997119,0.999948,0.996609,0.992730,0.992810,0.998927,...,0.996000,0.785423,0.066280,0.066280,-0.001140,-0.001140,-0.005847,-0.005847,-0.001865,-0.001865
hist_close,0.999726,0.999676,1.000000,0.319004,0.997436,0.999728,0.996926,0.993047,0.993128,0.999248,...,0.996317,0.785261,0.066302,0.066302,-0.001234,-0.001234,-0.005888,-0.005888,-0.001912,-0.001912
tick_vol,0.319086,0.319069,0.319004,1.000000,0.318091,0.319052,0.318044,0.317253,0.317408,0.317902,...,0.317855,0.225396,0.057761,0.057761,-0.356833,-0.356833,0.008332,0.008332,0.021261,0.021261
momentum_kama,0.997164,0.997119,0.997436,0.318091,1.000000,0.997166,0.999441,0.995660,0.995696,0.997103,...,0.998888,0.784031,0.066353,0.066353,-0.000863,-0.000863,-0.006425,-0.006425,-0.002601,-0.002601
others_cr,0.999999,0.999948,0.999728,0.319052,0.997166,1.000000,0.996656,0.992778,0.992859,0.998976,...,0.996047,0.785424,0.066328,0.066328,-0.001144,-0.001144,-0.005809,-0.005809,-0.001823,-0.001823
trend_ema_fast,0.996654,0.996609,0.996926,0.318044,0.999441,0.996656,1.000000,0.996211,0.996236,0.996574,...,0.999444,0.783599,0.066294,0.066294,-0.000617,-0.000617,-0.006521,-0.006521,-0.002646,-0.002646
trend_ema_slow,0.992777,0.992730,0.993047,0.317253,0.995660,0.992778,0.996211,1.000000,0.999968,0.993096,...,0.996756,0.781508,0.066281,0.066281,-0.001013,-0.001013,-0.007093,-0.007093,-0.002965,-0.002965
trend_ichimoku_a,0.992858,0.992810,0.993128,0.317408,0.995696,0.992859,0.996236,0.999968,1.000000,0.992998,...,0.996778,0.781551,0.066198,0.066198,-0.001093,-0.001093,-0.007264,-0.007264,-0.003030,-0.003030
trend_ichimoku_b,0.998974,0.998927,0.999248,0.317902,0.997103,0.998976,0.996574,0.993096,0.992998,1.000000,...,0.996052,0.785837,0.067673,0.067673,-0.001028,-0.001028,-0.004310,-0.004310,-0.001576,-0.001576


In [57]:
# Initialize the StandardScaler
sc = StandardScaler()

# Select the columns to be scaled and fit-transform them
columns_to_scale = ['tick_vol', 'others_cr', 'volume_vpt', 'volume_obv']
usd_df[columns_to_scale] = sc.fit_transform(usd_df[columns_to_scale])

# Get the description of the transformed columns
usd_df[columns_to_scale].describe().T

,count,mean,std,min,25%,50%,75%,max
tick_vol,100000.0,-1.023182e-16,1.000005,-1.116916,-0.683634,-0.305280,0.335830,10.942020
others_cr,100000.0,1.318767e-16,1.000005,-1.916359,-1.109996,0.398559,0.845057,1.887463
volume_vpt,100000.0,-2.728484e-17,1.000005,-1.991235,-0.757723,0.022483,0.620796,3.266767
volume_obv,100000.0,1.546141e-16,1.000005,-1.322085,-1.031109,0.012400,0.780053,2.061354


In [63]:
test = usd_df.close
train = usd_df.drop(columns=['close', 'date_time'])

X_train, X_test, y_train, y_test = train_test_split(
    train, test, test_size=0.05, shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(
    train, test, test_size=0.05, shuffle=False)

print(X_train.shape, y_train.shape, X_val.shape,
    y_val.shape, X_test.shape, y_test.shape)

(95000, 35) (95000,) (5000, 35) (5000,) (5000, 35) (5000,)


In [61]:
X = sm.add_constant(X_train)
estimator = sm.OLS(y_train, X)
usd_linear_model = estimator.fit()
print(usd_linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.025e+08
Date:                Fri, 16 Aug 2024   Prob (F-statistic):               0.00
Time:                        10:56:38   Log-Likelihood:             5.3198e+05
No. Observations:               88218   AIC:                        -1.064e+06
Df Residuals:                   88193   BIC:                        -1.064e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [62]:
usd_df.to_csv('data for modeling.csv', index=False)